# Recipies based bot powered by llamaindex, HuggingFace (for embedding model) and using AWS Bedrock

This notebook shows you how to use `ReAct` Agent over defined tools and a RAG pipeline with Claude 3 Sonnet. Key components used are -

- Reranker model (mixedbread-ai/mxbai-rerank-base-v1)
- Vector Database (AWS OpenSearch)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (mixedbread-ai/mxbai-embed-large-v1)
- Datasets ( Recipies - link , Calorie Count - link)


In [2]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch
%pip install llama-index-embeddings-huggingface


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.7.1
Uninstalling pydantic-2.7.1:
  Successfully uninstalled pydantic-2.7.1
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank

In [4]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ------------------------------------------------------------------------
# LlamaIndex - Amazon Bedrock


llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
#embed_model = BedrockEmbedding(model = "cohere.embed-english-v3")
embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

#BAAI/bge-large-en-v1.5
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()


Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.text_splitter = text_splitter


from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [5]:
import os
from os import getenv
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)
from llama_index.core import VectorStoreIndex, StorageContext

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "https://vpc-llamaindex-ank67ovnioagxtgqsqlq6iaqwa.us-west-2.es.amazonaws.com")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "gpt-index-huggingface-text-v1")

In [6]:
%pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank
postprocessor = SentenceTransformerRerank(
    model="mixedbread-ai/mxbai-rerank-large-v1", top_n=3
)

Note: you may need to restart the kernel to use updated packages.


In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
calorie_docs = SimpleDirectoryReader(input_dir="./recipies-data/calories").load_data(num_workers=8)

nodes = node_parser.get_nodes_from_documents(documents=calorie_docs,storage_context=storage_context )
base_nodes = text_splitter.get_nodes_from_documents(documents=calorie_docs,storage_context=storage_context )


# initialize an index using our sample data and the client we just created

from llama_index.core import VectorStoreIndex

calories_index = VectorStoreIndex(nodes)

calories_base_index = VectorStoreIndex(base_nodes)

#calories_index = VectorStoreIndex.from_documents(
#    documents=calorie_docs, storage_context=storage_context
#)

#calorie_engine = calories_index.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/llama_index/core/readers/file/base.py:538: UserWarning: Specified num_workers exceed number of CPUs in the system. Setting `num_workers` down to the maximum CPU count.
  warnings.warn(


In [9]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

calorie_engine = calories_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

window_response = calorie_engine.query(
    "How many calories in Egg and Eggnog?"
)
print(window_response)


#res = calorie_engine.query("How many calories in Egg and Eggnog")
#res.response

According to the context information provided:

Egg: 100g contains 97 calories
Eggnog: 100ml contains 88 calories


In [10]:
window_response = calorie_engine.query(
    "How many calories in Chicken and Chicken Breast?"
)
print(window_response)


According to the calorie information provided:

Chicken contains 219 calories per 100g.
Chicken Breast contains 172 calories per 100g.


In [11]:
from llama_index.core.node_parser import TokenTextSplitter
text_splitter = TokenTextSplitter()
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024
Settings.chunk_overlap=20
Settings.text_splitter = text_splitter

In [12]:


from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
recipies_index = VectorStoreIndex.from_documents(
    documents=recipies_docs, storage_context=storage_context
)

recipies_engine = recipies_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

#recipie_engine = recipies_index.as_query_engine(
#    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
 #   node_postprocessors=[
 #       MetadataReplacementPostProcessor(target_metadata_key="window")
 #   ],
#)



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f41f5c7f340>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f41f5a77670>


In [13]:
# run query
#query_engine = index_cohere.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])
res = recipies_engine.query("How can we make pasta?")
res.response

'Here are the steps to make fresh pasta from the context information provided:\n\n1. Make the pasta dough by mixing flour, eggs, and salt until a shaggy dough forms. Knead until smooth and elastic, then let rest for 1-2 hours.\n\n2. Divide the rested dough into pieces and flatten each piece into a rectangle. \n\n3. Use a pasta machine to gradually roll out each piece of dough, passing it through the rollers multiple times and adjusting to narrower settings until you reach the desired thickness, around 1/16 inch thick.\n\n4. Cut the rolled out pasta sheets into noodles, either by hand or using the cutting attachment on the pasta machine. \n\n5. Allow the cut noodles to dry slightly for at least 5 minutes before cooking.\n\n6. Cook the fresh pasta noodles in boiling salted water for around 2 minutes until tender.\n\nThe context provides detailed instructions for making fresh egg fettuccine noodles as well as a basic egg pasta dough that can be shaped into various pasta types. Following t

In [14]:
from llama_index.core.node_parser import TokenTextSplitter
text_splitter = TokenTextSplitter()
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024
Settings.chunk_overlap=20
Settings.text_splitter = text_splitter

In [15]:
# load data from calories datasaet and create an index
uber_docs = SimpleDirectoryReader(input_dir="./data/10k").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
uber_index = VectorStoreIndex.from_documents(
    documents=uber_docs, storage_context=storage_context
)

uber_engine = uber_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

In [16]:
# load data from calories datasaet and create an index
amazon_docs = SimpleDirectoryReader(input_dir="./data/amazon-10k").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
amazon_index = VectorStoreIndex.from_documents(
    documents=amazon_docs, storage_context=storage_context
)

amazon_engine = amazon_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

In [17]:
# load data from calories datasaet and create an index
google_docs = SimpleDirectoryReader(input_dir="./data/Google-10k").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
google_index = VectorStoreIndex.from_documents(
    documents=google_docs, storage_context=storage_context
)

google_engine = google_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

In [18]:
# load data from calories datasaet and create an index
movielens_docs = SimpleDirectoryReader(input_dir="./data/movielens").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



# initialize an index using our sample data and the client we just created
movielens_index = VectorStoreIndex.from_documents(
    documents=movielens_docs, storage_context=storage_context
)

movielens_engine = movielens_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])

In [43]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=calorie_engine,
        metadata=ToolMetadata(
            name="calorie_metadata",
            description=(
                "Provides information about calories across dairy, fruits and vegetables. You can use this tool to find out calories of different ingridients in a recipe."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=recipies_engine,
        metadata=ToolMetadata(
            name="recipies",
            description=(
                "Provides information about food recipies of various dishes. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber",
            description=(
                "Provides information about Uber's financial documents and SEC 10k filings. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=amazon_engine,
        metadata=ToolMetadata(
            name="amazon",
            description=(
                "Provides information about Amazons financial documents and SEC 10k filings. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=google_engine,
        metadata=ToolMetadata(
            name="google",
            description=(
                "Provides information about Google financial documents and SEC 10k filings. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=movielens_engine,
        metadata=ToolMetadata(
            name="movielens",
            description=(
                "Provides information about movies, plots and actors"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### With ReAct Agent

In [20]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill fish dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. To find options for a grilled fish dish with less than 500 calories, I will first need to look up calorie information for different types of fish and common ingredients used in grilled fish dishes.

Action: calorie_metadata
Action Input: {"input": "What are the calorie counts for common types of fish like salmon, tilapia, cod, etc?"}

Observation: Here are some typical calorie counts for common types of fish (per 3.5 oz serving, cooked):

Salmon: 155-200 calories
Tilapia: 94 calories  
Cod: 89 calories
Tuna: 109 calories
Halibut: 94 calories
Mahi Mahi: 90 calories

Thought: Okay, most types of fish are relatively low in calories, especially leaner white fish like cod, tilapia and halibut which have under 100 calories per 3.5 oz serving. Next I should look at common ingredients used in grilled fish dishes to see what low-calorie options there are.

Action: recipies
Action Input: {"input": "What are some common low-calorie ingredient

In [21]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("How many calories in sliced gouda cheese?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find the calorie information for sliced gouda cheese.
Action: calorie_metadata
Action Input: {'input': 'How many calories are in sliced gouda cheese?'}
Observation: According to the context information provided, 100g of sliced Gouda cheese contains 356 calories or 1495 kJ.
Thought: The calorie metadata tool provided the calorie information for sliced gouda cheese. I can now answer the original question.
Answer: 100g of sliced gouda cheese contains 356 calories.
100g of sliced gouda cheese contains 356 calories.


In [22]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill chicken dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is English. To find options for a grilled chicken dish with less than 500 calories, I will first need to look up calorie information for chicken and common ingredients used in grilled chicken dishes.

Action: calorie_metadata
Action Input: {"input": "How many calories are in 100g of boneless, skinless chicken breast?"}

Observation: According to the calorie metadata, 100g of boneless, skinless chicken breast contains approximately 165 calories.

Thought: To keep the total calories under 500, I should look for low-calorie ingredients to accompany the chicken. Vegetables are generally low in calories.

Action: calorie_metadata  
Action Input: {"input": "What are some low calorie vegetables I can use for a grilled chicken dish?"}

Observation: Some low calorie vegetable options for a grilled chicken dish include:

- Tomatoes (18 calories per 100g)
- Zucchini (17 calories per 100g) 
- Bell peppers (20 calories per 100g)
- Onions (40 calories per 10

In [23]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a morning breakfast smoothie which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is English. To find options for a low-calorie breakfast smoothie, I will need to use the calorie_metadata and recipies tools.
Action: calorie_metadata
Action Input: {'input': 'What are some low calorie ingredients I can use to make a breakfast smoothie?'}
Observation: For a low-calorie breakfast smoothie, you can use ingredients like:

- Almond milk or soy milk (41-45 calories per 100ml)
- Greek yogurt or low-fat yogurt
- Fresh or frozen fruits like berries, bananas, mangoes (generally low in calories)
- Leafy greens like spinach or kale (minimal calories)
- Chia seeds or flaxseeds (high in fiber and nutrients)
- Unsweetened cocoa powder or cinnamon for flavor
- Ice cubes to thicken the smoothie without adding calories

Avoid adding high-calorie ingredients like peanut butter, sweetened syrups, or full-fat milk/yogurt to keep the calorie count low. Using water or unsweetened plant-based milks as the liquid base can also help reduce calories.
Th

In [24]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("I want to make a cheesecake which has fruits as the topping and is not more than 2000 calories? Think Step by Step and recommend.")
print(str(response))

Thought: The current language of the user is: English. To find a cheesecake recipe with fruits as topping and under 2000 calories, I will first need to look up calorie information for cheesecake ingredients and toppings.
Action: calorie_metadata
Action Input: {'input': 'How many calories are in a typical cheesecake?'}
Observation: According to the calorie information provided, a typical cheesecake contains 321 calories per 100 grams.
Thought: Okay, that gives me a baseline for the cheesecake itself. Now I need to look at potential fruit toppings and their calorie counts to ensure the total is under 2000 calories.
Action: calorie_metadata
Action Input: {'input': 'What are the calorie counts for common fruit toppings like strawberries, blueberries, raspberries, etc. per 100 grams?'}
Observation: According to the calorie information provided:

Strawberries: 100g - 32 cal
Blueberries: 100g - 57 cal  
Raspberries: 100g - 52 cal
Blackberries: 100g - 43 cal
Cranberries: 100g - 46 cal
Thought:

In [25]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend me a cocktail drink which is based on Egg Nog? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is English. To recommend a cocktail drink based on eggnog and provide a calorie count, I will need to use the recipies and calorie_metadata tools.

Action: recipies
Action Input: {"input": "cocktail recipes containing eggnog"}

Observation: Here are some popular cocktail recipes that contain eggnog:

Eggnog Martini:
- 2 oz vodka
- 4 oz eggnog
- 1/2 oz coffee liqueur (like Kahlua)
- Nutmeg for garnish

Shake the vodka, eggnog and coffee liqueur with ice and strain into a chilled martini glass. Garnish with grated nutmeg.

Bourbon Eggnog:
- 2 oz bourbon
- 4 oz eggnog
- 1 oz milk or cream
- Cinnamon stick for garnish

Shake the bourbon, eggnog and milk/cream with ice and strain into a glass filled with fresh ice. Garnish with a cinnamon stick.

Rum Eggnog:
- 2 oz dark rum
- 4 oz eggnog
- Whipped cream and nutmeg for garnish

Shake the rum and eggnog with ice and strain into a glass. Top with whipped cream and grated nutmeg.

Action: calorie_metada

In [26]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog based cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is English. To recommend eggnog-based cocktails and provide calorie counts, I will need to use the recipies and calorie_metadata tools.

Action: recipies
Action Input: {"input": "eggnog cocktail recipes"}

Observation: Here are some popular eggnog cocktail recipes:

Eggnog White Russian:
- 2 oz vodka
- 1 oz coffee liqueur 
- 4 oz eggnog
- Nutmeg for garnish

Spiked Eggnog:
- 4 oz eggnog
- 1 oz rum
- 1 oz brandy
- Cinnamon stick for garnish  

Eggnog Martini:
- 3 oz eggnog
- 2 oz vodka
- 1 oz coffee liqueur
- Nutmeg for garnish

Bourbon Eggnog:
- 4 oz eggnog 
- 2 oz bourbon
- Cinnamon stick for garnish

Action: calorie_metadata
Action Input: {"input": "calories in eggnog, vodka, coffee liqueur, rum, brandy, bourbon"}

Observation: - 1 cup eggnog: 343 calories
- 1 oz vodka: 64 calories  
- 1 oz coffee liqueur (e.g. Kahlua): 69 calories
- 1 oz rum: 65 calories
- 1 oz brandy: 73 calories
- 1 oz bourbon: 69 calories

Thought: I now have enough infor

In [27]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is English. To recommend eggnog cocktails and provide calorie counts, I will need to use the recipies and calorie_metadata tools.

Action: recipies
Action Input: {"input": "eggnog cocktail recipes"}

Observation: Here are some popular eggnog cocktail recipes:

Classic Eggnog Cocktail:
- 4 oz eggnog
- 2 oz rum or brandy
- Nutmeg for garnish

Peppermint Eggnog White Russian:
- 2 oz eggnog
- 1 oz vodka
- 1 oz coffee liqueur
- 1/4 tsp peppermint extract
- Crushed candy cane for garnish

Spiked Eggnog Latte:
- 4 oz eggnog
- 2 oz coffee
- 1 oz Irish cream liqueur
- Whipped cream and cinnamon for garnish

Eggnog Martini:
- 3 oz eggnog
- 2 oz vodka
- 1 oz coffee liqueur
- Cinnamon sugar for rim

Action: calorie_metadata
Action Input: {"input": "calories in eggnog"}

Observation: One cup (240 ml) of eggnog contains approximately:

- Calories: 343
- Fat: 19 g
- Carbs: 38 g  
- Protein: 10 g

The calorie and nutrient content can vary based on the specific

In [28]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. To recommend a good Indian curry dish and provide calorie counts, I will need to use the recipies and calorie_metadata tools.
Action: recipies
Action Input: {'input': 'Recommend a good Indian curry dish with ingredients and instructions'}
Observation: Here is a delicious Indian prawn moilee (shrimp curry with coconut) recipe:

Ingredients:
- 6 green chilies
- 1/4 cup vegetable oil 
- 2 tsp mustard seeds
- 30 fresh curry leaves
- 2 cups sliced onions
- 1 tbsp garlic paste
- 1 tbsp ginger paste  
- 2 tsp salt
- 1 tsp black pepper
- 1 1/4 tsp turmeric powder
- 1/4 cup fresh ginger matchsticks
- 2 cups coconut milk
- 1 cup coconut cream
- 24 large prawns/shrimp
- 1 1/2 cups chopped tomatoes
- Lemon wedges for serving

Instructions:

1. Heat oil in a pan and temper with mustard seeds and curry leaves. 
2. Add the sliced onions and sauté until translucent.
3. Add garlic, ginger pastes, salt, pepper, turmeric and ginger matchsticks. Cook 

In [29]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish which is low on calories and good on fiber? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. To recommend a low-calorie, high-fiber Indian curry dish, I will need information on calorie counts and fiber content of different ingredients.
Action: calorie_metadata
Action Input: {'input': 'What are some low-calorie and high-fiber ingredients commonly used in Indian curries?'}
Observation: Based on the context information provided, some low-calorie and high-fiber ingredients commonly used in Indian curries include:

Lentils (dal) - Lentils are low in calories (230 cal per 100g) and high in fiber. Different varieties like brown lentils, red lentils, and yellow lentils are used in Indian curries.

Okra - Okra is low in calories (33 cal per 100g) and a good source of fiber. It is used in curries like bhindi masala.

Spinach - Spinach is very low in calories (23 cal per 100g) and provides fiber. It is used in palak paneer and other vegetable curries.

Cauliflower - Cauliflower is low in calories (25 cal per 100g) and provides fiber

In [44]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("WHat were key challenges faced by Amazon and Google in 2022?")
print(str(response))

Thought: The current language of the user is: English. To answer this question about challenges faced by Amazon and Google in 2022, I will need to use the amazon and google tools to look for relevant information in their financial documents and SEC filings.
Action: amazon
Action Input: {'input': 'What were the key challenges faced by Amazon in 2022 based on their financial documents and SEC filings?'}
Observation: Based on the financial documents and SEC filings provided, some of the key challenges faced by Amazon in 2022 appear to be:

1. Unfavorable impact from foreign exchange rates: Amazon's Q1 2023 guidance anticipates an unfavorable impact of approximately 210 basis points from foreign exchange rates on their net sales.

2. Macroeconomic headwinds: The guidance mentions factors like fluctuations in global economic and geopolitical conditions, customer demand and spending (including recessionary fears), inflation, and interest rates as potential challenges impacting their results.

In [32]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you compare plot of movies from Tom Hanks and Tom Cruise? Give movie names as you talk about the plot")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me compare the plots of movies from Tom Hanks and Tom Cruise.
Action: movielens
Action Input: {'input': 'Provide information about movies, plots and actors for Tom Hanks and Tom Cruise'}
Observation: Here are some details about movies, plots, and actors for Tom Hanks and Tom Cruise based on the provided context:

Tom Hanks:
- A Hologram for the King (2016) - Comedy/Drama film where Hanks plays Alan Clay, an American businessman trying to sell technology in Saudi Arabia while dealing with bureaucracy and personal issues.
- Big (1988) - Fantasy/Comedy where Hanks plays Josh Baskin, a 13-year-old boy whose wish to be "big" is granted overnight, forcing him to navigate the adult world.

Tom Cruise: 
- Knight and Day (2010) - Action/Comedy film where Cruise plays a fugitive who gets caught up in a deadly adventure with Cameron Diaz's character. Their survival depends on the battle between truth and trust ami

In [45]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Compare the investments made by Uber and Lyft")
print(str(response))

Thought: The current language of the user is: English. To compare the investments made by Uber and Lyft, I will need information about their financial documents and SEC filings.
Action: uber
Action Input: {'input': "What information is available about Uber's investments and financial details?"}
Observation: Based on the context information provided, here are some key details about Uber's investments and financial details:

- Uber had $4.2 billion in cash and cash equivalents as of December 31, 2022, down slightly from $4.3 billion at the end of 2021.

- Uber held $103 million in short-term investments and $1.6 billion in restricted investments as of the end of 2022. 

- Uber's total investments, including equity method investments, were valued at $5.3 billion as of December 31, 2022, down from $12.6 billion at the end of 2021.

- Uber had $9.3 billion in long-term debt, net of current portion, as of the end of 2022, relatively unchanged from 2021.

- Uber incurred a net loss of $9.1 bi

In [46]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Who had more revenue in 2022 , Amazon or Google?")
print(str(response))

Thought: The current language of the user is: English. To answer this question, I need to look at the financial information for Amazon and Google.
Action: amazon
Action Input: {'input': "What was Amazon's revenue in 2022?"}
Observation: According to the financial information provided, Amazon's total net sales (revenue) for the year 2022 were $513,983 million.

The breakdown of net sales by segment is as follows:

- North America: $315,880 million 
- International: $118,007 million
- AWS (Amazon Web Services): $80,096 million

Overall, Amazon's consolidated net sales increased by 9% in 2022 compared to the previous year 2021.
Thought: I now have Amazon's 2022 revenue, but still need Google's 2022 revenue to compare and answer the question.
Action: google
Action Input: {'input': "What was Google's revenue in 2022?"}
Observation: According to the financial results provided, Google's total revenues for the year ended December 31, 2022 were $282.836 billion.
Thought: I now have the 2022 rev

In [47]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Who had more revenue in 2023 , Amazon or Google?")
print(str(response))

Thought: The current language of the user is: English. To answer this question, I need to look at the financial information for Amazon and Google, specifically their revenue numbers for 2023.
Action: amazon
Action Input: {'input': "What was Amazon's revenue in 2023?"}
Observation: Unfortunately, the provided context does not explicitly state Amazon's revenue for the full year 2023. However, it does provide guidance for Amazon's expected net sales in the first quarter of 2024, which are projected to be between $138.0 billion and $143.5 billion, representing an 8% to 13% increase compared to the first quarter of 2023.
Thought: Okay, the Amazon tool did not directly provide Amazon's 2023 revenue, but gave a projection for Q1 2024 revenue compared to Q1 2023. To fully answer the question, I also need Google's 2023 revenue.
Action: google
Action Input: {'input': "What was Google's revenue in 2023?"}
Observation: According to the financial results provided, Google's total revenues for the ye

In [48]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("What was Amazon revenue in 2023 and can you name some movies from Tom Hanks?")
print(str(response))

Thought: The current language of the user is: English. To answer this question, I need to use the amazon and movielens tools.

Action: amazon
Action Input: {"input": "What was Amazon's revenue in 2023?"}

Observation: I'm afraid I don't have information about Amazon's revenue in the future year of 2023. As an AI assistant without access to future data, I can only provide information from the company's historical financial reports and filings up to the present day.

Thought: Okay, the amazon tool cannot provide future revenue data for 2023. I will try the movielens tool to get information about Tom Hanks movies.

Action: movielens
Action Input: {"input": "Can you provide some examples of movies starring Tom Hanks?"}

Observation: Here are some popular movies starring Tom Hanks:

- Forrest Gump (1994)
- Saving Private Ryan (1998) 
- Cast Away (2000)
- Catch Me If You Can (2002)
- The Terminal (2004)
- The Da Vinci Code (2006)
- Captain Phillips (2013)
- Sully (2016)
- A Beautiful Day in 

In [50]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill fish dish which has less than 500 calories? think step by step")
print(str(response))

Thought: The current language of the user is: English. To find options for a grilled fish dish with less than 500 calories, I will first need to look up calorie information for different types of fish and common ingredients used in grilled fish dishes.
Action: calorie_metadata
Action Input: {'input': 'What are the calorie counts for common types of fish like salmon, tuna, tilapia, cod, etc?'}
Observation: Unfortunately the provided context does not contain calorie information for common types of fish like salmon, tuna, tilapia, cod, etc. The context appears to be a large dataset of calorie counts for various food items, but it does not seem to include entries specifically for different types of fish.
Thought: Okay, the calorie_metadata tool does not have specific calorie information for different types of fish. I will try looking for grilled fish recipes and their calorie counts instead.
Action: recipies
Action Input: {'input': 'What are some healthy grilled fish recipes with calorie c